In [1]:
import pandas as pd
import numpy as np
import chess.pgn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [71]:
data = pd.read_csv('stockfish.csv', sep=',')
print(data.shape)

(50000, 2)


In [23]:
pgn = open('data.pgn')
y_white = []
y_black = []
for i in range(25000):
    game = chess.pgn.read_game(pgn)
    y_white.append(game.headers['WhiteElo'])
    y_black.append(game.headers['BlackElo'])

In [24]:
copy_y_white = y_white.copy()
copy_y_black = y_black. copy()

In [25]:
y_white = copy_y_white.copy()
y_black = copy_y_black.copy()

In [26]:
matches = data.values[:, 1]
X_white = []
X_black = []
for match in matches[:25000]:
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) < 3:
        del y_white[len(X_white)]
        del y_black[len(X_white)]
    else:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        X_white.append([white.min(), white.max(), white.mean(), white.std()])
        X_black.append([black.min(), black.max(), black.mean(), black.std()])
        
    

In [27]:
print(len(y_white), len(X_white))

24972 24972


In [28]:
for i in range(len(X_white)):
    if np.isnan(X_white[i]).any() or np.isnan(X_black[i]).any():
        print(X_black[i])

In [29]:
X_train, X_test, y_train, y_test = train_test_split((X_white+X_black), (y_white+y_black), test_size=0.3, random_state=1)

In [30]:
len(X_train), len(X_test)

(34960, 14984)

In [31]:
y_train = np.array(list(map(int, y_train)))
y_train = y_train.reshape((len(y_train), ))
y_train

array([1981, 2615, 1955, ..., 1716, 1883, 2571])

In [32]:
np.isnan(X_train).any(), np.isnan(y_train).any()

(False, False)

In [33]:
m = LinearRegression()
m.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
y_test = np.array(list(map(int, y_test)))

In [35]:
y_pred = m.predict(X_test)
mean_absolute_error(y_pred, y_test)

210.04339391734118

In [36]:
m.coef_

array([-0.02814548,  0.01471871,  0.25746884, -0.18637048])

# Send result

In [111]:
m2 = LinearRegression()
m2.fit((X_white+X_black), (y_white+y_black))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [65]:
X_test = []
for match in matches[25000:]:
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) > 1:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        X_test.append([white.min(), white.max(), white.mean(), white.std()])
        X_test.append([black.min(), black.max(), black.mean(), black.std()])
    else:
        X_test.append(X_test[-2])
        X_test.append(X_test[-1])

In [66]:
len(X_test)

50000

In [67]:
y_pred2 = m2.predict(X_test)

In [68]:
len(list(range(25001, 50001))), len(y_pred2[::2]), len(y_pred2[1::2])

(25000, 25000, 25000)

In [69]:
df = pd.DataFrame({'Event': list(range(25001, 50001)), 'WhiteElo': y_pred2[::2], 'BlackElo':y_pred2[1::2]})
df.to_csv("result1.csv", index=None, columns=['Event', 'WhiteElo', 'BlackElo'])

In [70]:
min(y_pred2)

1159.2920880034371

# using deltas as feature

In [44]:
matches_data = pd.read_csv('stockfish.csv', sep=',')
delta_data = pd.read_csv('deltas2.csv', sep=',')
y = pd.read_csv('y.csv', sep=',')
y_white = list(y.values[:, 0])
y_black = list(y.values[:, 1])
result = list(y.values[:, 2]) + list(y.values[:, 3])

In [2]:
pgn = open('data.pgn')
y_white = []
y_black = []
result = []
for i in range(25000):
    game = chess.pgn.read_game(pgn)
    y_white.append(game.headers['WhiteElo'])
    y_black.append(game.headers['BlackElo'])
    result.append(game.headers['Result'])
for i in range(25000):
    game = chess.pgn.read_game(pgn)
    result.append(game.headers['Result'])

In [45]:
df = pd.DataFrame({'y_white': y_white, 'y_black': y_black, 'result_train': result[:25000], 'result_test': result[25000:]})
df.to_csv("y.csv", index=None, columns=['y_white', 'y_black', 'result_train', 'result_test'])

In [5]:
copy_y_white = y_white.copy()
copy_y_black = y_black. copy()
len(y_white)

25000

In [23]:
y_white = copy_y_white.copy()
y_black = copy_y_black.copy()

In [46]:
le = LabelEncoder()
le.fit(result)
le.classes_

array(['0-1', '1-0', '1/2-1/2'],
      dtype='<U7')

In [47]:
tr = le.transform(result)
enc = OneHotEncoder()
enc.fit(tr.reshape(len(tr), 1))
encoded_result = enc.transform(tr.reshape(len(tr), 1)).toarray()
print(encoded_result.shape)

(50000, 3)


In [48]:
train_result = list(encoded_result[:25000])
type(train_result)

list

In [9]:
matches = matches_data.values[:, 1]
X_white = []
X_black = []
for match, delta in zip(matches[:25000], delta_data.values[:25000, :]):
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) < 3:
        del y_white[len(X_white)]
        del y_black[len(X_white)]
        del train_result[len(X_white)]
    else:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        white_deltas = np.fromstring(delta[1], sep=' ', dtype=int)
        black_deltas = -1 * np.fromstring(delta[2], sep=' ', dtype=int)
        X_white.append([white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std()])
        X_black.append([black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std()])

In [11]:
len((X_white+X_black)), len((y_white+y_black)), len(train_result) * 2

(49944, 49944, 49944)

In [10]:
X = np.hstack((np.array((X_white+X_black)), np.array(train_result + train_result)))
X.shape

(49944, 12)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, (y_white+y_black), test_size=0.3, random_state=1)
y_train = np.array(list(map(int, y_train)))
y_train = y_train.reshape((len(y_train), ))
m = LinearRegression()
m.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
y_test = np.array(list(map(int, y_test)))
y_pred = m.predict(X_test)
mean_absolute_error(y_pred, y_test)

207.64845459994129

In [48]:
m.coef_

array([ -1.51083489e-02,   1.48039792e-02,   4.55559719e-02,
        -1.13146295e-01,   3.41224976e-01,  -2.33122035e-02,
        -2.91428078e-02,   1.11001635e+00,   2.67150048e-02,
        -2.42038092e+01,  -1.47033298e+01,   3.89071390e+01])

# Send 2

In [49]:
m2 = LinearRegression()
m2.fit(X, (y_white+y_black))
X[0]

array([-26.        ,  70.        ,  20.73684211,  26.58223013,
        18.        , -27.        ,  30.        ,  -3.57894737,
        16.51893902,   0.        ,   0.        ,   1.        ])

In [50]:
m2.coef_

array([ -1.78156189e-02,   1.60928986e-02,   5.70899350e-02,
        -1.26586054e-01,   3.39506590e-01,  -2.28834593e-02,
        -2.44862513e-02,   1.01212258e+00,   1.16985740e-02,
        -2.36691758e+01,  -1.30305132e+01,   3.66996890e+01])

In [16]:
test_result = list(encoded_result[25000:])
type(test_result)
X.shape, np.hstack((np.array(X_test), np.array(test_result + test_result))).shape

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [17]:
X_test = []
for match, delta in zip(matches[25000:], delta_data.values[25000:, :]):
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) > 1:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        white_deltas = np.fromstring(delta[1], sep=' ', dtype=int)
        black_deltas = -1 * np.fromstring(delta[2], sep=' ', dtype=int)
        X_test.append([white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std()])
        X_test.append([black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std()])
    else:
        X_test.append(X_test[-2])
        X_test.append(X_test[-1])

In [52]:
y_pred2 = m2.predict(np.hstack((np.array(X_test), np.array(test_result + test_result))))

In [53]:
df = pd.DataFrame({'Event': list(range(25001, 50001)), 'WhiteElo': y_pred2[::2], 'BlackElo':y_pred2[1::2]})
df.to_csv("result2.csv", index=None, columns=['Event', 'WhiteElo', 'BlackElo'])

# Random Forest

In [2]:
from sklearn.ensemble import RandomForestRegressor

In [4]:
regr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
regr.get_params().keys()

dict_keys(['bootstrap', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [13]:
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
y_test = np.array(list(map(int, y_test)))
y_pred = regr.predict(X_test)
mean_absolute_error(y_pred, y_test)

198.75929905361016

# Send Random Forest

In [21]:
regr = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=0)
regr.fit(X, (y_white+y_black))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [22]:
y_pred2 = regr.predict(np.hstack((np.array(X_test), np.array(test_result + test_result))))

In [23]:
df = pd.DataFrame({'Event': list(range(25001, 50001)), 'WhiteElo': y_pred2[::2], 'BlackElo':y_pred2[1::2]})
df.to_csv("result3.csv", index=None, columns=['Event', 'WhiteElo', 'BlackElo'])

# early game, mid game, late game

In [133]:
matches_data = pd.read_csv('stockfish.csv', sep=',')
delta_data = pd.read_csv('deltas2.csv', sep=',')
y = pd.read_csv('y.csv', sep=',')
y_white = list(y.values[:, 0])
y_black = list(y.values[:, 1])
result = list(y.values[:, 2]) + list(y.values[:, 3])

In [134]:
le = LabelEncoder()
le.fit(result)
le.classes_
tr = le.transform(result)
enc = OneHotEncoder()
enc.fit(tr.reshape(len(tr), 1))
encoded_result = enc.transform(tr.reshape(len(tr), 1)).toarray()
print(encoded_result.shape)
train_result = list(encoded_result[:25000])
type(train_result)

(50000, 3)


list

In [135]:
matches = matches_data.values[:, 1]
X_white = []
X_black = []
for match, delta in zip(matches[:25000], delta_data.values[:25000, :]):
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) < 9:
        del y_white[len(X_white)]
        del y_black[len(X_white)]
        del train_result[len(X_white)]
    else:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        white_deltas = np.fromstring(delta[1], sep=' ', dtype=int)
        black_deltas = -1 * np.fromstring(delta[2], sep=' ', dtype=int)
        if len(white_deltas)//3 == 0:
            print(black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].mean())
        X_white.append([white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std(), white_deltas[:len(white_deltas)//3].mean(), white_deltas[:len(white_deltas)//3].std(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].mean(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].std(), white_deltas[2 * len(white_deltas)//3:].mean(), white_deltas[2 * len(white_deltas)//3:].std(), black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std(), black_deltas[:len(black_deltas)//3].mean(), black_deltas[:len(black_deltas)//3].std(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].mean(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].std(), black_deltas[2 * len(black_deltas)//3:].mean(), black_deltas[2 * len(black_deltas)//3:].std()])
        X_black.append([black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std(), black_deltas[:len(black_deltas)//3].mean(), black_deltas[:len(black_deltas)//3].std(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].mean(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].std(), black_deltas[2 * len(black_deltas)//3:].mean(), black_deltas[2 * len(black_deltas)//3:].std(), white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std(), white_deltas[:len(white_deltas)//3].mean(), white_deltas[:len(white_deltas)//3].std(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].mean(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].std(), white_deltas[2 * len(white_deltas)//3:].mean(), white_deltas[2 * len(white_deltas)//3:].std()])

In [136]:
len((X_white+X_black)), len((y_white+y_black)), len(train_result) * 2

(49900, 49900, 49900)

In [137]:
X = np.hstack((np.array((X_white+X_black)), np.array(train_result + train_result)))
X.shape

(49900, 33)

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, (y_white+y_black), test_size=0.05, random_state=1)
y_train = np.array(list(map(int, y_train)))
y_train = y_train.reshape((len(y_train), ))

In [143]:
regr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [144]:
y_test = np.array(list(map(int, y_test)))
y_pred = regr.predict(X_test)
mean_absolute_error(y_pred, y_test)

192.10443889840732

# Send last modified features

In [145]:
regr = RandomForestRegressor(n_estimators=500, min_samples_split=50, min_samples_leaf=5, max_features=1.0, max_depth=9, bootstrap=True, random_state=0)
regr.fit(X, (y_white+y_black))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features=1.0, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=50,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [146]:
X_test = []
for match, delta in zip(matches[25000:], delta_data.values[25000:, :]):
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) > 9:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        white_deltas = np.fromstring(delta[1], sep=' ', dtype=int)
        black_deltas = -1 * np.fromstring(delta[2], sep=' ', dtype=int)
        X_test.append([white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std(), white_deltas[:len(white_deltas)//3].mean(), white_deltas[:len(white_deltas)//3].std(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].mean(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].std(), white_deltas[2 * len(white_deltas)//3:].mean(), white_deltas[2 * len(white_deltas)//3:].std(), black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std(), black_deltas[:len(black_deltas)//3].mean(), black_deltas[:len(black_deltas)//3].std(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].mean(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].std(), black_deltas[2 * len(black_deltas)//3:].mean(), black_deltas[2 * len(black_deltas)//3:].std()])
        X_test.append([black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std(), black_deltas[:len(black_deltas)//3].mean(), black_deltas[:len(black_deltas)//3].std(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].mean(), black_deltas[len(black_deltas)//3:2 * len(black_deltas)//3].std(), black_deltas[2 * len(black_deltas)//3:].mean(), black_deltas[2 * len(black_deltas)//3:].std(), white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std(), white_deltas[:len(white_deltas)//3].mean(), white_deltas[:len(white_deltas)//3].std(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].mean(), white_deltas[len(white_deltas)//3:2 * len(white_deltas)//3].std(), white_deltas[2 * len(white_deltas)//3:].mean(), white_deltas[2 * len(white_deltas)//3:].std()])
    else:
        X_test.append(X_test[-2])
        X_test.append(X_test[-1])

In [147]:
test_result = list(encoded_result[25000:])
type(test_result)
X.shape, np.hstack((np.array(X_test), np.array(test_result + test_result))).shape

((49900, 33), (50000, 33))

In [148]:
y_pred2 = regr.predict(np.hstack((np.array(X_test), np.array(test_result + test_result))))

In [149]:
df = pd.DataFrame({'Event': list(range(25001, 50001)), 'WhiteElo': y_pred2[::2], 'BlackElo':y_pred2[1::2]})
df.to_csv("result7.csv", index=None, columns=['Event', 'WhiteElo', 'BlackElo'])

# Use grid to estimate params

In [16]:
from sklearn import model_selection, grid_search, metrics

In [93]:
gr = RandomForestRegressor(bootstrap=True, random_state=0)
gr.get_params().keys()

dict_keys(['bootstrap', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [94]:
parameters_grid = {
    'max_depth': list(range(7, 10, 2)),
    'min_samples_split': list(range(30, 51, 10)),
    'min_samples_leaf': list(range(5, 10, 2)),
}

In [39]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

In [95]:
grid_cv = grid_search.GridSearchCV(gr, parameters_grid, scoring='neg_mean_absolute_error')

In [96]:
%%time
grid_cv.fit(X, (y_white+y_black))

Wall time: 2min 16s


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [7, 9], 'min_samples_split': [30, 40, 50], 'min_samples_leaf': [5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [98]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

-196.90821882023863
{'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 50}


# Split match into 5 parts

In [109]:
matches = matches_data.values[:, 1]
X_white = []
X_black = []
for match, delta in zip(matches[:25000], delta_data.values[:25000, :]):
    match = match.replace('NA', '')
    parsed = np.fromstring(match, sep=' ', dtype=int)
    if len(parsed) < 10:
        del y_white[len(X_white)]
        del y_black[len(X_white)]
        del train_result[len(X_white)]
    else:
        white = parsed[0::2]
        black = -1 * parsed[1::2]
        white_deltas = np.fromstring(delta[1], sep=' ', dtype=int)
        black_deltas = -1 * np.fromstring(delta[2], sep=' ', dtype=int)
        if len(white_deltas)//5 == 0:
            print(black_deltas[len(black_deltas)//5:2 * len(black_deltas)//5].mean())
        X_white.append([white.min(), white.max(), white.mean(), white.std(), np.median(white), white_deltas.min(), white_deltas.max(), white_deltas.mean(), white_deltas.std(), white_deltas[:len(white_deltas)//5].mean(), white_deltas[len(white_deltas)//5:2 * len(white_deltas)//5].mean(), white_deltas[2 * len(white_deltas)//5:3*len(white_deltas)//5].mean(), white_deltas[3 * len(white_deltas)//5:4*len(white_deltas)//5].mean(), white_deltas[4 * len(white_deltas)//5:].mean()])
        X_black.append([black.min(), black.max(), black.mean(), black.std(), np.median(black), black_deltas.min(), black_deltas.max(), black_deltas.mean(), black_deltas.std(), black_deltas[:len(black_deltas)//5].mean(), black_deltas[len(black_deltas)//5:2 * len(black_deltas)//5].mean(), black_deltas[2 * len(black_deltas)//5:3*len(black_deltas)//5].mean(), black_deltas[3 * len(black_deltas)//5:4*len(black_deltas)//5].mean(), black_deltas[4 * len(black_deltas)//5:].mean()])